In [1]:
DATASET_data="rec.mu2e.CRV_wideband_cosmics.CRVWB-000-008-000.root"
DATASET_MC_v1="nts.mu2e.CRV_wideband_cosmics-mc.config_0012_v1.root"
DATASET_MC_v2="nts.mu2e.CRV_wideband_cosmics-mc.config_0012_v2.root"
DATASET_MC_v7 = "nts.mu2e.WidebandCRV_v7.MDC2020ae.root"

In [2]:
import time
import os
import numpy as np
import gc
from mu2etools import *
from mu2etools import wideband

hep.style.use('ATLAS')

In [3]:
runs = [1720, 1721, 1722, 1723, 1724, 1727, 1728, 1729, 1730, 1731, 1732, 1738, 2004, 2005, 2006, 2007, 2008, 2009, 2015, 2024, 2025, 2026, 2027, 2028]
processor = wideband.DataProcessor(usexroot=True, treename='run', runlist=runs, userunlist=True, remove=False)
filelist_data = processor.getFilelist(DATASET_data)
#append "run" at the end of each file name
filelist_data = ["{}{}".format(i, ":run") for i in filelist_data]

In [4]:
processor = wideband.DataProcessor(usexroot=True, treename='CrvWidebandTest/WidebandTree', runlist=[1738], userunlist=False, remove=False)

filelist_mc_v7 = processor.getFilelist(DATASET_MC_v7)
#append "run" at the end of each file name
filelist_mc_v7 = ["{}{}".format(i, ":CrvWidebandTest/run") for i in filelist_mc_v7]

filelist_mc_v2 = processor.getFilelist(DATASET_MC_v2)
#append "run" at the end of each file name
filelist_mc_v2 = ["{}{}".format(i, ":CrvWidebandTest/run") for i in filelist_mc_v2]

filelist_mc_v1 = processor.getFilelist(DATASET_MC_v1)
#append "run" at the end of each file name
#filelist_mc = ["{}{}".format(i, ":CrvWidebandTest/WidebandTree") for i in filelist_mc]
filelist_mc_v1 = ["{}{}".format(i, ":CrvWidebandTest/run") for i in filelist_mc_v1]

In [5]:
# Real Wideband data

# List of variable to import
varlist=['PEs', 'trackIntercept', 'trackChi2', 'trackPoints', 'trackPEs', 
         'runNumber', 'subrunNumber', 'spillNumber', 'eventNumber']
varlist_keep = ['PEs_testlayers', 'runNumber', 'subrunNumber', 'spillNumber', 'eventNumber']
#varlist_keep = ['PEs_testlayers', 'trackIntercept', 'trackChi2NDF', 'trackPoints', 'trackPEs', 
#         'PEsPerHit', 'runNumber', 'subrunNumber', 'spillNumber', 'eventNumber']

ar_skim_list_cleanup_data = []
ar_skim_list_169_data = []

histbins = np.linspace(2,400,800)
hPEs_data, bins = np.histogram([], bins=histbins)

allevents = 0
cutevents_cleanup = 0
cutevents_169 = 0

allfiles = 0
filesloaded = 0

for idx, filename in enumerate(filelist_data):
    if idx%100 == 0:
        print("Processing file: %s"%filename)
    allfiles += 1
    try:
        with uproot.open(filename) as file:
            for ar in file.iterate(step_size="10MB", 
                                           filter_name=varlist, 
                                           report=False, library='ak'):
                allevents += len(ar["eventNumber"])

                ar["PEs"] = ak.where(ar["PEs"] >= 5, ar["PEs"], 0)

                FEB0_0 = (ak.sum(ar["PEs"][:,0,8:32], axis=-1)>10)
                FEB0_1 = (ak.sum(ar["PEs"][:,0,32:64], axis=-1)>10)

                FEB1_0 = (ak.sum(ar["PEs"][:,1,4:28], axis=-1)>10) #0:32
                FEB1_1 = (ak.sum(ar["PEs"][:,1,36:60], axis=-1)>10)

                FEB2_0 = (ak.sum(ar["PEs"][:,2,4:28], axis=-1)>10)
                FEB2_1 = (ak.sum(ar["PEs"][:,2,36:60], axis=-1)>10) #32:64

                FEB3_0 = (ak.sum(ar["PEs"][:,3,8:32], axis=-1)>10)
                FEB3_1 = (ak.sum(ar["PEs"][:,3,32:64], axis=-1)>10)

                FEB4_0 = (ak.sum(ar["PEs"][:,4,0:32], axis=-1)>10)
                FEB4_1 = (ak.sum(ar["PEs"][:,4,32:64], axis=-1)>10)

                FEB5_0 = (ak.sum(ar["PEs"][:,5,0:32], axis=-1)>10)
                FEB5_1 = (ak.sum(ar["PEs"][:,5,32:64], axis=-1)>10)

                FEB6_0 = (ak.sum(ar["PEs"][:,6,0:32], axis=-1)>10)
                FEB6_1 = (ak.sum(ar["PEs"][:,6,32:64], axis=-1)>10)

                FEB7_0 = (ak.sum(ar["PEs"][:,7,8:32], axis=-1)>10)
                FEB7_1 = (ak.sum(ar["PEs"][:,7,32:64], axis=-1)>10)

                nTrigHits = ak.values_astype(FEB0_0, "int") + ak.values_astype(FEB0_1, "int") + \
                            ak.values_astype(FEB3_0, "int") + ak.values_astype(FEB3_1, "int") + \
                            ak.values_astype(FEB4_0, "int") + ak.values_astype(FEB4_1, "int") + \
                            ak.values_astype(FEB5_0, "int") + ak.values_astype(FEB5_1, "int") + \
                            ak.values_astype(FEB6_0, "int") + ak.values_astype(FEB6_1, "int") + \
                            ak.values_astype(FEB7_0, "int") + ak.values_astype(FEB7_1, "int")

                hPEsRun, bins = np.histogram(ak.flatten(ar["PEs"][:,1,0:64]), bins=histbins)
                hPEs_data = np.add(hPEs_data, hPEsRun)
                hPEsRun, bins = np.histogram(ak.flatten(ar["PEs"][:,2,0:64]), bins=histbins)
                hPEs_data = np.add(hPEs_data, hPEsRun)

                ar['trackChi2NDF'] = ak.where(ar['trackPoints'] > 2, ar['trackChi2'] / (ar['trackPoints'] - 2), -999)
                ar['PEsPerHit'] = ak.where(ar['trackPoints'] > 2, ar['trackPEs'] / (ar['trackPoints']), -999)
                ar["PEs_testlayers"] = ar["PEs"][:,np.arange(1,3)]

                ar_cleanup = ar[(nTrigHits >=11) & (ar["trackIntercept"] < -200) & (ar["trackIntercept"] > -700) & (ar["trackPEs"] < 2600) & (ar["trackChi2NDF"] < 25) & (ar["trackChi2NDF"] > 0)]
                ar_169 = ar[FEB1_0 & FEB2_1]
                cutevents_cleanup += len(ar_cleanup["eventNumber"])
                cutevents_169 += len(ar_169["eventNumber"])

                # Append to the list
                ar_skim_list_cleanup_data.append(ar_cleanup[varlist_keep])
                ar_skim_list_169_data.append(ar_169[varlist_keep])

                filesloaded += 1
    except:
        continue
    
        
print("{0} files loaded out of {1}".format(filesloaded, allfiles))
print("Total events: {0}".format(allevents))
print("Events after cleanup cuts: {0}".format(cutevents_cleanup))
print("Events after module 169 triggering: {0}".format(cutevents_169))

Processing file: root://fndcadoor.fnal.gov:1094/mu2e/tape/phy-rec/rec/mu2e/CRV_wideband_cosmics/CRVWB-000-008-000/root/fc/22/rec.mu2e.CRV_wideband_cosmics.CRVWB-000-008-000.001720_000.root:run


/home/thoroho/.conda/envs/Mu2eCRV/lib/python3.10/site-packages/awkward/_backends.py:57: RuntimeWarning: divide by zero encountered in divide
  return getattr(ufunc, method)(*args, **kwargs)
/home/thoroho/.conda/envs/Mu2eCRV/lib/python3.10/site-packages/awkward/_backends.py:57: RuntimeWarning: invalid value encountered in divide
  return getattr(ufunc, method)(*args, **kwargs)


475 files loaded out of 73
Total events: 9507173
Events after cleanup cuts: 2718855
Events after module 169 triggering: 4428035
